In [1]:
%%html
<h2>TensorFlow による実装 ：手書きの数字の画像データ判定<br>
活性化関数：relu<br>
隠れ層：3層<br>
</h2>

In [2]:
import numpy as np
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(0)
tf.set_random_seed(1234)

'''
データの生成
'''
mnist = datasets.fetch_mldata('MNIST original', data_home='.')

n = len(mnist.data)
N = 10000  # MNISTの一部を使う
train_size = 0.8
# シャッフルされたインデックスの取得 → 10000 個選択
indices = np.random.permutation(range(n))[:N]

X = mnist.data[indices]
y = mnist.target[indices]
# 10 * 10 の単位行列
Y = np.eye(10)[y.astype(int)]  # 1-of-K 表現に変換

# 訓練データと試験データの分割
X_train, X_test, Y_train, Y_test =\
    train_test_split(X, Y, train_size=train_size)

'''
モデル設定
'''
n_in = len(X[0])  # 784
n_hidden = 200
n_out = len(Y[0])  # 10

x = tf.placeholder(tf.float32, shape=[None, n_in])
# t は真の分類結果
t = tf.placeholder(tf.float32, shape=[None, n_out])

# 入力層 - 隠れ層
W0 = tf.Variable(tf.truncated_normal([n_in, n_hidden], stddev=0.01))
b0 = tf.Variable(tf.zeros([n_hidden]))
h0 = tf.nn.relu(tf.matmul(x, W0) + b0)

# 隠れ層 - 隠れ層
W1 = tf.Variable(tf.truncated_normal([n_hidden, n_hidden], stddev=0.01))
b1 = tf.Variable(tf.zeros([n_hidden]))
h1 = tf.nn.relu(tf.matmul(h0, W1) + b1)

W2 = tf.Variable(tf.truncated_normal([n_hidden, n_hidden], stddev=0.01))
b2 = tf.Variable(tf.zeros([n_hidden]))
h2 = tf.nn.relu(tf.matmul(h1, W2) + b2)

W3 = tf.Variable(tf.truncated_normal([n_hidden, n_hidden], stddev=0.01))
b3 = tf.Variable(tf.zeros([n_hidden]))
h3 = tf.nn.relu(tf.matmul(h2, W3) + b3)

# 隠れ層 - 出力層
W4 = tf.Variable(tf.truncated_normal([n_hidden, n_out], stddev=0.01))
b4 = tf.Variable(tf.zeros([n_out]))
y = tf.nn.softmax(tf.matmul(h3, W4) + b4)

# 学習率
learning_rate = 0.01

## 誤差関数テンソル
#  t : 真の分類結果　と y : 近似予測結果 との誤差関数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(t * tf.log(y),
                               reduction_indices=[1]))

## 学習 Operationクラス
#   最急降下法による誤差最小化を行う
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

## 学習の評価
#   x,t のテンソルで表現された学習結果, 予測成功したのか True/False の列挙で表現する
#   correct_prediction_val:  [ True  True  True ..., False  True  True]
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))

# 学習結果をさらに、float32 にキャストして平均値を計算
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

'''
モデル学習
'''
epochs = 50
batch_size = 200

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# バッチサイズ
n_batches = (int)(N * train_size)

# t.eval -> Tensor クラスの run 実行のためのショートカット
# t.eval() is a shortcut for calling tf.get_default_session().run(t)

for epoch in range(epochs):
    X_, Y_ = shuffle(X_train, Y_train)

    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size

        sess.run(train_step, feed_dict={
            x: X_[start:end],
            t: Y_[start:end]
        })

    # 訓練データに対する学習の進み具合を出力
    loss = cross_entropy.eval(session=sess, feed_dict={
        x: X_,
        t: Y_
    })
    acc = accuracy.eval(session=sess, feed_dict={
        x: X_,
        t: Y_
    })
    print('epoch:', epoch, ' loss:', loss, ' accuracy:', acc)

'''
予測精度の評価
'''
correct_prediction_val = correct_prediction.eval(session=sess, feed_dict={
    x: X_test,
    t: Y_test
})
print('correct_prediction_val: ', correct_prediction_val)
accuracy_rate = accuracy.eval(session=sess, feed_dict={
    x: X_test,
    t: Y_test
})
print('accuracy: ', accuracy_rate)

epoch: 0  loss: 2.30208  accuracy: 0.11525
epoch: 1  loss: 2.30157  accuracy: 0.11525
epoch: 2  loss: 2.30101  accuracy: 0.11525
epoch: 3  loss: 2.30035  accuracy: 0.11525
epoch: 4  loss: 2.2995  accuracy: 0.11525
epoch: 5  loss: 2.29836  accuracy: 0.11525
epoch: 6  loss: 2.2967  accuracy: 0.11525
epoch: 7  loss: 2.29401  accuracy: 0.115375
epoch: 8  loss: 2.28902  accuracy: 0.132375
epoch: 9  loss: 2.27752  accuracy: 0.295375
epoch: 10  loss: 2.23875  accuracy: 0.372375
epoch: 11  loss: 2.02113  accuracy: 0.3435
epoch: 12  loss: 1.39772  accuracy: 0.464875
epoch: 13  loss: 1.57368  accuracy: 0.38325
epoch: 14  loss: 1.2345  accuracy: 0.507
epoch: 15  loss: 0.791202  accuracy: 0.741625
epoch: 16  loss: 0.661055  accuracy: 0.81
epoch: 17  loss: 0.573629  accuracy: 0.818375
epoch: 18  loss: 0.504778  accuracy: 0.85225
epoch: 19  loss: 0.467585  accuracy: 0.855625
epoch: 20  loss: 0.453164  accuracy: 0.859625
epoch: 21  loss: 0.383353  accuracy: 0.89075
epoch: 22  loss: 0.381399  accuracy